# TikTok Shop API Integration - FIXED VERSION

## Tổng quan dự án:
- **Mục đích**: Tích hợp API TikTok Shop để lấy dữ liệu seller, products, orders
- **API Version**: 202309 (Latest stable)
- **Authentication**: OAuth 2.0 + HMAC-SHA256 signature
- **Base URL**: `https://open-api.tiktokglobalshop.com`
- **Token URL**: `https://auth.tiktok-shops.com/api/v2/token/get`

## Các bước thực hiện:
1. **Cấu hình credentials** (App Key, App Secret)
2. **Tạo Authorization URL** cho user consent
3. **Lấy Authorization Code** từ callback URL
4. **Đổi code thành Access Token**
5. **Gọi API lấy dữ liệu** (Seller, Products, Orders)
6. **Xử lý và hiển thị dữ liệu**

## Lưu ý quan trọng:
- Code này đã được sửa lỗi theo **Official TikTok Shop API Documentation**
- Signature algorithm đã được cập nhật đúng format
- Endpoints và parameters đã được kiểm tra
- Hỗ trợ cả domestic và cross-border shops

## BƯỚC 1: Import Libraries và Cấu hình

In [1]:
# Import required libraries
import requests
import json
import time
import hmac
import hashlib
import pandas as pd
import openpyxl
from datetime import datetime, timedelta
from urllib.parse import urlencode, parse_qs, urlparse

print("Libraries imported successfully!")


Libraries imported successfully!


## BƯỚC 2: Cấu hình API Credentials

In [2]:
# ===== CẤUHÌNH API CREDENTIALS =====
# THAY ĐỔI CÁC GIÁ TRỊ SAU ĐÂY:

APP_KEY = "6h2cosrovhjab"          # Lấy từ TikTok Developer Portal
APP_SECRET = "e66f20d50276eb96c52c483ea8606714216f817e"    # Lấy từ TikTok Developer Portal
REDIRECT_URI = "https://facolospickleball.com/" # Website đã đăng ký với TikTok

# API Configuration - FIXED TOKEN ENDPOINT
BASE_URL = "https://open-api.tiktokglobalshop.com"
TOKEN_URL = "https://auth.tiktok-shops.com/api/v2/token/get"  # Official token endpoint
AUTH_URL = "https://auth.tiktok-shops.com/oauth/authorize"

# UPDATED SCOPES: 8 quyền đã được chấp thuận (2 quyền đang chờ duyệt)
SCOPES = [
    "seller.finance.info",                    # Thông tin tài chính
    "seller.fulfillment.basic",              # Thông tin fulfillment
    "seller.global_product.category.info",   # Danh mục sản phẩm toàn cầu
    "seller.shop.info",                      # Thông tin shop
    "seller.logistics",                      # Thông tin logistics  
    "seller.order.info",                     # Thông tin đơn hàng
    "seller.product.basic",                  # Thông tin sản phẩm
    "seller.return_refund.basic",            # Thông tin trả hàng
    "seller.authorization.info"              # THÊM MỚI: Authorization info - Cần để lấy shop_cipher
    # seller.customer_service - Đang chờ duyệt
    # seller.promotion.info - Đang chờ duyệt
]

# Validation
def validate_config():
    missing = []
    if APP_KEY == "YOUR_APP_KEY_HERE":
        missing.append("APP_KEY")
    if APP_SECRET == "YOUR_APP_SECRET_HERE":
        missing.append("APP_SECRET")
    if REDIRECT_URI == "YOUR_WEBSITE_URL_HERE":
        missing.append("REDIRECT_URI")
    return missing

# Check configuration
missing_fields = validate_config()

if missing_fields:
    print("CONFIGURATION INCOMPLETE!")
    print(f"Missing fields: {', '.join(missing_fields)}")
    print("\nTO FIX:")
    print("1. Go to TikTok Developer Portal: https://partner.tiktokshop.com/developer")
    print("2. Select your app > Basic Information")
    print("3. Copy App Key and App Secret")
    print("4. Update REDIRECT_URI with your registered website")
    print("5. Replace the values above and run this cell again")
else:
    print("CONFIGURATION COMPLETE!")
    print(f"App Key: {APP_KEY[:15]}...")
    print(f"Redirect URI: {REDIRECT_URI}")
    print(f"Scopes: {', '.join(SCOPES)}")
    print(f"Token URL: {TOKEN_URL}")
    print("\nREADY FOR NEXT STEP!")
    print("NOTE: Scopes updated to match granted permissions")

CONFIGURATION COMPLETE!
App Key: 6h2cosrovhjab...
Redirect URI: https://facolospickleball.com/
Scopes: seller.finance.info, seller.fulfillment.basic, seller.global_product.category.info, seller.shop.info, seller.logistics, seller.order.info, seller.product.basic, seller.return_refund.basic, seller.authorization.info
Token URL: https://auth.tiktok-shops.com/api/v2/token/get

READY FOR NEXT STEP!
NOTE: Scopes updated to match granted permissions


## BƯỚC 3: Generate Authorization URL

In [3]:
def generate_authorization_url():
    """
    Tạo Authorization URL để user cấp quyền
    """
    # Check config first
    if validate_config():
        print("Please complete configuration in Step 2 first!")
        return None
    
    # Authorization parameters
    auth_params = {
        'app_key': APP_KEY,
        'response_type': 'code',
        'redirect_uri': REDIRECT_URI,
        'scope': ','.join(SCOPES),
        'state': f"state_{int(time.time())}"  # Random state for security
    }
    
    # Build authorization URL
    auth_url = f"{AUTH_URL}?{urlencode(auth_params)}"
    
    print("AUTHORIZATION URL GENERATED:")
    print("="*80)
    print(auth_url)
    print("="*80)
    print("\nNEXT STEPS:")
    print("1. Copy the URL above")
    print("2. Open in browser")
    print("3. Login to TikTok Shop account")
    print("4. Grant permissions")
    print("5. Copy the callback URL you get redirected to")
    print("6. Paste callback URL in Step 4")
    
    return {
        'auth_url': auth_url,
        'state': auth_params['state']
    }

# Generate authorization URL
auth_result = generate_authorization_url()

if auth_result:
    auth_url = auth_result['auth_url']
    state = auth_result['state']
    print(f"\nState: {state}")
    print("Keep this state value to verify callback!")

AUTHORIZATION URL GENERATED:
https://auth.tiktok-shops.com/oauth/authorize?app_key=6h2cosrovhjab&response_type=code&redirect_uri=https%3A%2F%2Ffacolospickleball.com%2F&scope=seller.finance.info%2Cseller.fulfillment.basic%2Cseller.global_product.category.info%2Cseller.shop.info%2Cseller.logistics%2Cseller.order.info%2Cseller.product.basic%2Cseller.return_refund.basic%2Cseller.authorization.info&state=state_1756104208

NEXT STEPS:
1. Copy the URL above
2. Open in browser
3. Login to TikTok Shop account
4. Grant permissions
5. Copy the callback URL you get redirected to
6. Paste callback URL in Step 4

State: state_1756104208
Keep this state value to verify callback!


## BƯỚC 4: Parse Authorization Code từ Callback URL

In [5]:
# ===== PASTE CALLBACK URL HERE =====
# Thay đổi URL bên dưới bằng callback URL bạn nhận được:
CALLBACK_URL = "https://facolospickleball.com/?app_key=6h2cosrovhjab&code=ROW_po4WgwAAAADWKV8GfMJYq-chSgZLuOlNONNTPO1sN_9R2BMo_JCMpwUUFMG-nBh5q8KSx6ZQvWZjffS2CUAOFPYl3rwFoWze5-hDsNsqEIP85LqcJY5tog&locale=vi-VN&shop_region=VN&state=state_1756104208"

def parse_authorization_code(callback_url):
    """
    Parse authorization code từ callback URL
    """
    if callback_url == "PASTE_YOUR_CALLBACK_URL_HERE":
        print("Please paste your callback URL above!")
        return None
    
    try:
        # Parse URL
        parsed_url = urlparse(callback_url)
        query_params = parse_qs(parsed_url.query)
        
        print(f"Parsing URL: {callback_url}")
        print(f"Query params: {query_params}")
        
        # Extract authorization code
        if 'code' in query_params:
            auth_code = query_params['code'][0]
            callback_state = query_params.get('state', [None])[0]
            
            print("AUTHORIZATION CODE EXTRACTED SUCCESSFULLY!")
            print(f"Code: {auth_code[:20]}...")
            print(f"State: {callback_state}")
            
            # Verify state if available
            if 'state' in locals() and callback_state != state:
                print("WARNING: State mismatch! Possible security issue.")
            
            return auth_code
        
        elif 'error' in query_params:
            error = query_params['error'][0]
            error_description = query_params.get('error_description', ['No description'])[0]
            
            print(f"AUTHORIZATION ERROR: {error}")
            print(f"Description: {error_description}")
            return None
        
        else:
            print("No authorization code or error found in callback URL")
            return None
            
    except Exception as e:
        print(f"Error parsing callback URL: {e}")
        return None

# Parse authorization code
auth_code = parse_authorization_code(CALLBACK_URL)

if auth_code:
    print("\nREADY FOR TOKEN EXCHANGE!")
else:
    print("\nCannot proceed without authorization code")

Parsing URL: https://facolospickleball.com/?app_key=6h2cosrovhjab&code=ROW_po4WgwAAAADWKV8GfMJYq-chSgZLuOlNONNTPO1sN_9R2BMo_JCMpwUUFMG-nBh5q8KSx6ZQvWZjffS2CUAOFPYl3rwFoWze5-hDsNsqEIP85LqcJY5tog&locale=vi-VN&shop_region=VN&state=state_1756104208
Query params: {'app_key': ['6h2cosrovhjab'], 'code': ['ROW_po4WgwAAAADWKV8GfMJYq-chSgZLuOlNONNTPO1sN_9R2BMo_JCMpwUUFMG-nBh5q8KSx6ZQvWZjffS2CUAOFPYl3rwFoWze5-hDsNsqEIP85LqcJY5tog'], 'locale': ['vi-VN'], 'shop_region': ['VN'], 'state': ['state_1756104208']}
AUTHORIZATION CODE EXTRACTED SUCCESSFULLY!
Code: ROW_po4WgwAAAADWKV8G...
State: state_1756104208

READY FOR TOKEN EXCHANGE!


## BƯỚC 5: Core Functions - Signature Generation

In [6]:
def generate_signature(app_secret, path, params, body=""):
    """
    FIXED: Generate HMAC-SHA256 signature theo Official TikTok Shop Documentation
    
    Steps theo docs:
    1. Extract query params excluding 'sign' and 'access_token'
    2. Sort alphabetically by key
    3. Concatenate in {key}{value} format (NO = or &)
    4. Prepend request path 
    5. Append body if provided
    6. Wrap with app_secret
    7. HMAC-SHA256 encode
    """
    # Step 1: Extract all query parameters excluding sign and access_token
    filtered_params = {k: str(v) for k, v in params.items() 
                      if k not in ['sign', 'access_token']}
    
    # Step 2: Sort parameters alphabetically by key
    sorted_keys = sorted(filtered_params.keys())
    
    # Step 3: Concatenate in {key}{value} format (NO = or &)
    param_string = ''.join([f"{key}{filtered_params[key]}" for key in sorted_keys])
    
    # Step 4: Start with request path + params
    sign_string = f"{path}{param_string}"
    
    # Step 5: Append body if provided (for POST requests)
    if body:
        if isinstance(body, dict):
            # Convert dict to JSON string without spaces
            body_string = json.dumps(body, separators=(',', ':'), sort_keys=True)
        else:
            body_string = str(body)
        sign_string += body_string
    
    # Step 6: Wrap with app_secret (app_secret + string + app_secret)
    wrapped_string = f"{app_secret}{sign_string}{app_secret}"
    
    # Debug output
    print(f"Signature Debug:")
    print(f"   Path: {path}")
    print(f"   Filtered params: {filtered_params}")
    print(f"   Param string: {param_string}")
    print(f"   Body: {body}")
    print(f"   Sign string: {sign_string}")
    print(f"   Wrapped: {wrapped_string[:50]}...")
    
    # Step 7: HMAC-SHA256 encode
    signature = hmac.new(
        app_secret.encode('utf-8'),
        wrapped_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()
    
    print(f"   Generated signature: {signature}")
    
    return signature

print("Signature generation function loaded!")

Signature generation function loaded!


## BƯỚC 6: Exchange Authorization Code for Access Token

In [7]:
def get_access_token(app_key, app_secret, auth_code):
    """
    FIXED: Exchange authorization code for access token
    Using OFFICIAL DOCUMENTATION (202309): GET request with query parameters
    """
    print("Exchanging authorization code for access token...")
    
    # Use configured endpoint
    token_url = TOKEN_URL
    
    # Query parameters for token exchange (according to official docs)
    params = {
        'app_key': app_key,
        'app_secret': app_secret,
        'auth_code': auth_code,
        'grant_type': 'authorized_code'  # Note: 'authorized_code' not 'authorization_code'
    }
    
    print(f"Token URL: {token_url}")
    print(f"Request params: {params}")
    
    try:
        # GET request with query parameters (according to official docs)
        response = requests.get(
            token_url,
            params=params,  # Query parameters
            timeout=30
        )
        
        print(f"Response Status: {response.status_code}")
        print(f"Raw Response: {response.text[:300]}...")
        
        # Try to parse JSON response
        try:
            result = response.json()
            print(f"Parsed Response: {json.dumps(result, indent=2)}")
            
            # Check for success (code=0 according to documentation)
            if result.get('code') == 0:
                data = result.get('data', {})
                print("SUCCESS! Access token received!")
                return {
                    'success': True,
                    'access_token': data.get('access_token'),
                    'refresh_token': data.get('refresh_token'),
                    'expires_in': data.get('access_token_expire_in'),
                    'refresh_expires_in': data.get('refresh_token_expire_in'),
                    'scope': data.get('scope'),
                    'open_id': data.get('open_id'),
                    'seller_name': data.get('seller_name'),
                    'seller_base_region': data.get('seller_base_region'),
                    'user_type': data.get('user_type'),
                    'request_id': result.get('request_id'),
                    'full_response': result
                }
            else:
                # Error response format according to docs
                error_code = result.get('code', 'unknown')
                error_msg = result.get('message', 'Unknown error')
                print(f"API Error {error_code}: {error_msg}")
                
                return {
                    'success': False,
                    'error': f'API Error {error_code}',
                    'message': error_msg,
                    'code': error_code,
                    'request_id': result.get('request_id'),
                    'full_response': result
                }
                
        except json.JSONDecodeError as json_error:
            print(f"JSON Decode Error: {json_error}")
            print(f"Raw response text: {response.text}")
            return {
                'success': False,
                'error': f'JSON Decode Error: {json_error}',
                'raw_response': response.text,
                'status_code': response.status_code
            }
            
    except Exception as e:
        print(f"Exception: {e}")
        return {
            'success': False,
            'error': f'Exception: {e}'
        }

# Exchange code for token
if 'auth_code' in locals() and auth_code:
    print("STARTING TOKEN EXCHANGE...")
    token_result = get_access_token(APP_KEY, APP_SECRET, auth_code)
    
    if token_result.get('success'):
        ACCESS_TOKEN = token_result['access_token']
        REFRESH_TOKEN = token_result['refresh_token']
        SHOP_CIPHER = token_result.get('shop_cipher')  # For cross-border shops
        
        print(f"\nTOKEN EXCHANGE SUCCESSFUL!")
        print(f"Access Token: {ACCESS_TOKEN[:30]}...")
        print(f"Refresh Token: {REFRESH_TOKEN[:30]}...")
        print(f"Expires in: {token_result['expires_in']} seconds")
        print(f"Seller: {token_result['seller_name']}")
        print(f"Region: {token_result['seller_base_region']}")
        print(f"User Type: {token_result['user_type']}")
        print(f"Request ID: {token_result['request_id']}")
        print("\nREADY FOR API CALLS!")
    else:
        print(f"\nTOKEN EXCHANGE FAILED!")
        print(f"Error: {token_result.get('error')}")
        print(f"Message: {token_result.get('message')}")
        print(f"Code: {token_result.get('code')}")
        print(f"Request ID: {token_result.get('request_id')}")
        if 'raw_response' in token_result:
            print(f"Raw Response: {token_result['raw_response'][:500]}...")
else:
    print("No authorization code available. Please complete Step 4 first!")

STARTING TOKEN EXCHANGE...
Exchanging authorization code for access token...
Token URL: https://auth.tiktok-shops.com/api/v2/token/get
Request params: {'app_key': '6h2cosrovhjab', 'app_secret': 'e66f20d50276eb96c52c483ea8606714216f817e', 'auth_code': 'ROW_po4WgwAAAADWKV8GfMJYq-chSgZLuOlNONNTPO1sN_9R2BMo_JCMpwUUFMG-nBh5q8KSx6ZQvWZjffS2CUAOFPYl3rwFoWze5-hDsNsqEIP85LqcJY5tog', 'grant_type': 'authorized_code'}
Response Status: 200
Raw Response: {"code":0,"message":"success","data":{"access_token":"ROW_KF0BZAAAAABMn2tP_U1z56VGExv3UNNiC5JX15XXZZE2p87uwh5Kx5v6vOenf_jUir4e9LF4SHrIIEEBDWBhTOlRPUSzkuyWqqkHsWbyREw3kvnXS6dGw2-CSvuyl1xhFI-UL0Wu8WI","access_token_expire_in":1756709142,"refresh_token":"ROW_Gl_kJgAAAADfgvAXKIs3kXfphGClhkRg5YpSvVSOBgLZ...
Parsed Response: {
  "code": 0,
  "message": "success",
  "data": {
    "access_token": "ROW_KF0BZAAAAABMn2tP_U1z56VGExv3UNNiC5JX15XXZZE2p87uwh5Kx5v6vOenf_jUir4e9LF4SHrIIEEBDWBhTOlRPUSzkuyWqqkHsWbyREw3kvnXS6dGw2-CSvuyl1xhFI-UL0Wu8WI",
    "access_toke

In [8]:
# BƯỚC 6A: Kiểm tra thời hạn refresh token từ API response
def check_refresh_token_expiry():
    """Kiểm tra thời hạn refresh token từ API response thực tế"""
    
    if 'token_result' in globals() and token_result.get('success'):
        refresh_expires_in = token_result.get('refresh_expires_in')
        
        if refresh_expires_in:
            days = refresh_expires_in / (24 * 60 * 60)
            hours = refresh_expires_in / 3600
            
            print(f"REFRESH TOKEN EXPIRY INFO:")
            print(f"   Seconds: {refresh_expires_in:,}")
            print(f"   Hours: {hours:,.1f}")
            print(f"   Days: {days:.1f}")
            print(f"   Expires at: {datetime.fromtimestamp(time.time() + refresh_expires_in)}")
        else:
            print("No refresh token expiry info in response")
    else:
        print("No successful token exchange found")

# Chạy kiểm tra
check_refresh_token_expiry()

REFRESH TOKEN EXPIRY INFO:
   Seconds: 4,876,620,543
   Hours: 1,354,616.8
   Days: 56442.4
   Expires at: 2180-03-07 22:37:51.189679


## BƯỚC 7: Get Shop Cipher (seller.authorization.info)

In [10]:
# BƯỚC 7: Lấy Shop Cipher từ Authorization API
print("GETTING SHOP_CIPHER FROM AUTHORIZATION API...")
print("="*60)

def get_shop_cipher_from_auth():
    """Lấy shop_cipher từ authorization/202309/shops endpoint"""
    
    if 'ACCESS_TOKEN' not in globals() or not ACCESS_TOKEN:
        print("ACCESS_TOKEN not available")
        return None
    
    # Authorization shops endpoint theo documentation
    endpoint = '/authorization/202309/shops'
    
    # Parameters cho API call
    params = {
        'app_key': APP_KEY,
        'timestamp': str(int(time.time())),
        'sign_method': 'hmac_sha256'
    }
    
    # Generate signature
    filtered = {k: str(v) for k, v in params.items() if k not in ['sign', 'access_token']}
    sorted_keys = sorted(filtered.keys())
    param_string = ''.join(f"{k}{filtered[k]}" for k in sorted_keys)
    sign_string = f"{endpoint}{param_string}"
    wrapped = f"{APP_SECRET}{sign_string}{APP_SECRET}"
    params['sign'] = hmac.new(APP_SECRET.encode(), wrapped.encode(), hashlib.sha256).hexdigest()
    
    # Headers
    headers = {
        'x-tts-access-token': ACCESS_TOKEN,
        'Content-Type': 'application/json'
    }
    
    # API call
    url = f"https://open-api.tiktokglobalshop.com{endpoint}"
    
    try:
        response = requests.get(url, params=params, headers=headers, timeout=15)
        
        print(f"Status: {response.status_code}")
        print(f"Response: {response.text[:300]}...")
        
        if response.status_code == 200:
            result = response.json()
            
            if result.get('code') == 0:
                shops = result.get('data', {}).get('shops', [])
                
                if shops:
                    shop = shops[0]  # Lấy shop đầu tiên
                    shop_cipher = shop.get('cipher')
                    
                    print(f"SUCCESS! Shop details:")
                    print(f"   Shop ID: {shop.get('id')}")
                    print(f"   Shop Name: {shop.get('name')}")
                    print(f"   Region: {shop.get('region')}")
                    print(f"   Seller Type: {shop.get('seller_type')}")
                    print(f"   Shop Cipher: {shop_cipher}")
                    
                    return shop_cipher
                else:
                    print(f"No shops found in response")
            else:
                print(f"API Error: {result.get('message')}")
        else:
            print(f"HTTP Error: {response.status_code}")
            
    except Exception as e:
        print(f"Exception: {e}")
    
    return None

# Lấy shop_cipher
SHOP_CIPHER = get_shop_cipher_from_auth()

if SHOP_CIPHER:
    print(f"\nSHOP_CIPHER obtained: {SHOP_CIPHER}")
    print("Ready for Order Detail API!")
else:
    print(f"\nCould not get shop_cipher")

print("="*60)

GETTING SHOP_CIPHER FROM AUTHORIZATION API...
Status: 200
Response: {"code":0,"data":{"shops":[{"cipher":"ROW_lXdGlQAAAAAEW7t8sE8nkXPLwspQvX45","code":"VNLCXLWASG","id":"7495842618397395583","name":"Facolos Pickleball","region":"VN","seller_type":"LOCAL"}]},"message":"Success","request_id":"20250825144925DAC02247762633084ADF"}...
SUCCESS! Shop details:
   Shop ID: 7495842618397395583
   Shop Name: Facolos Pickleball
   Region: VN
   Seller Type: LOCAL
   Shop Cipher: ROW_lXdGlQAAAAAEW7t8sE8nkXPLwspQvX45

SHOP_CIPHER obtained: ROW_lXdGlQAAAAAEW7t8sE8nkXPLwspQvX45
Ready for Order Detail API!


## BƯỚC 8: Get Order Data (seller.order.info)

### Mục đích:
- **Lấy dữ liệu đơn hàng** từ TikTok Shop API
- **Sử dụng 2-step workflow**: Search Orders → Get Order Details
- **Yêu cầu**: ACCESS_TOKEN (Bước 6) + SHOP_CIPHER (Bước 7)

### Workflow:
```
Step 8.1: Search Orders → Lấy danh sách Order IDs
Step 8.2: Get Details → Lấy thông tin chi tiết từng đơn hàng
Step 8.3: Execute & Store → Chạy workflow và lưu kết quả
```

### Kết quả:
- **Global Variable**: `REAL_ORDER_DETAILS` chứa order data
- **Ready for**: Phân tích (Bước 9) và Export (Bước 10)

### BƯỚC 8.1: Search Orders (Lấy Order IDs)

#### Mục đích:
- **Tìm kiếm đơn hàng** theo điều kiện filter (status, ngày tạo)
- **Trả về danh sách Order IDs** để query chi tiết ở bước tiếp theo
- **Không lấy full data** - chỉ lấy IDs để tối ưu performance

#### API Specification:
- **Endpoint**: `/order/202309/orders/search`
- **Method**: POST
- **Authentication**: ACCESS_TOKEN + SHOP_CIPHER
- **Body**: Filter conditions (order_status, create_time_range)

#### Logic hoạt động:
1. **Setup query parameters** (app_key, timestamp, shop_cipher)
2. **Define filter body** (status UNPAID, last 30 days)
3. **Generate HMAC signature** với body content
4. **POST request** với signed parameters
5. **Extract order IDs** từ response

In [56]:
def search_orders_for_ids():
    """Step 1: Search orders để lấy order IDs - FIXED SIGNATURE"""
    
    if 'ACCESS_TOKEN' not in globals() or not ACCESS_TOKEN:
        print("ACCESS_TOKEN not available")
        return []
    
    if 'SHOP_CIPHER' not in globals() or not SHOP_CIPHER:
        print("SHOP_CIPHER not available")
        return []
    
    print("Step 1: Searching orders to get IDs...")
    
    # Search orders endpoint
    path = '/order/202309/orders/search'
    current_timestamp = str(int(time.time()))
    
    # Query parameters
    query_params = {
        'app_key': APP_KEY,
        'timestamp': current_timestamp,
        'shop_cipher': SHOP_CIPHER,
        'page_size': 20,
        'sort_order': 'DESC',
        'sort_field': 'create_time',
        'sign_method': 'hmac_sha256'
    }
    
    # Request body - MỞ RỘNG FILTER ĐỂ TÌM NHIỀU ĐơN HÀNG HƠN
    request_body = {
        # Thử nhiều status thay vì chỉ UNPAID
        # 'order_status': 'UNPAID',  # Bỏ filter này để lấy tất cả status
        'create_time_ge': int(time.time()) - (90 * 24 * 60 * 60),  # 90 days ago (thay vì 30)
        'create_time_lt': int(time.time())  # current time
    }
    
    # Serialize body ONCE and reuse for both signing and sending
    body_string = json.dumps(request_body, separators=(',', ':'), sort_keys=True)

    # Use the same exact string for signature
    signature = generate_signature(APP_SECRET, path, query_params, body_string)
    query_params['sign'] = signature
    
    # Headers
    headers = {
        'x-tts-access-token': ACCESS_TOKEN,
        'content-type': 'application/json'
    }
    
    try:
        url = f"https://open-api.tiktokglobalshop.com{path}"
        
        print(f"Making POST request to: {url}")
        
        response = requests.post(
            url,
            params=query_params,
            data=body_string,  # use serialized body to match signature
            headers=headers,
            timeout=15
        )
        
        print(f"Response Status: {response.status_code}")
        print(f"Response Text: {response.text[:500]}...")
        
        if response.status_code == 200:
            result = response.json()
            if result.get('code') == 0:
                orders = result.get('data', {}).get('orders', [])
                order_ids = [order.get('id') for order in orders if order.get('id')]
                
                print(f"SUCCESS! Found {len(orders)} orders")
                print(f"Order IDs: {order_ids[:5]}...")                
                return order_ids
            else:
                print(f"API Error {result.get('code')}: {result.get('message')}")
                return []
        else:
            print(f"HTTP Error {response.status_code}")
            try:
                error_data = response.json()
                print(f"   Error code: {error_data.get('code')}")
                print(f"   Error message: {error_data.get('message')}")
            except:
                print(f"   Raw response: {response.text}")
            return []
            
    except Exception as e:
        print(f"Exception in search: {e}")
        return []

### BƯỚC 8.2: Get Order Details (Lấy Chi tiết đơn hàng)

#### Mục đích:
- **Lấy thông tin đầy đủ** của các đơn hàng từ Order IDs
- **Trả về complete data**: payment, shipping, products, buyer info
- **Tối đa 10 orders** mỗi lần gọi (API limitation)

#### API Specification:
- **Endpoint**: `/order/202309/orders`
- **Method**: GET
- **Authentication**: ACCESS_TOKEN + SHOP_CIPHER
- **Query Params**: `ids=123,456,789` (comma-separated Order IDs)

#### Logic hoạt động:
1. **Validate Order IDs** từ Step 8.1
2. **Setup query parameters** với Order IDs
3. **Generate HMAC signature** (no body for GET)
4. **GET request** với signed parameters
5. **Return full order objects** với tất cả thông tin chi tiết 

In [57]:
def get_order_details_with_ids(order_ids):
    """Step 2: Get order details với order IDs - FIXED SIGNATURE"""
    
    if not order_ids:
        print("No order IDs provided")
        return []
    
    print(f"Step 2: Getting details for {len(order_ids)} orders...")
    
    # Order detail endpoint
    path = '/order/202309/orders'
    current_timestamp = str(int(time.time()))
    
    # Query parameters với order IDs
    query_params = {
        'app_key': APP_KEY,
        'timestamp': current_timestamp,
        'shop_cipher': SHOP_CIPHER,
        'ids': ','.join(order_ids[:10])  # Chỉ lấy tối đa 10 đơn hàng
    }
    
    # SỬ DỤNG FUNCTION generate_signature() ĐÃ ĐỊNH NGHĨA
    signature = generate_signature(APP_SECRET, path, query_params, body="")
    query_params['sign'] = signature
    
    # Headers
    headers = {
        'x-tts-access-token': ACCESS_TOKEN,
        'content-type': 'application/json'
    }
    
    try:
        url = f"https://open-api.tiktokglobalshop.com{path}"
        
        print(f"Making GET request to: {url}")
        
        response = requests.get(
            url,
            params=query_params,
            headers=headers,
            timeout=15
        )
        
        print(f"Response Status: {response.status_code}")
        print(f"Response Text: {response.text[:500]}...")
        
        if response.status_code == 200:
            result = response.json()
            if result.get('code') == 0:
                orders = result.get('data', {}).get('orders', [])
                print(f"SUCCESS! Retrieved {len(orders)} detailed orders")
                
                if orders:
                    print("Detailed orders:")
                    for order in orders[:3]:
                        print(f"   • ID: {order.get('id')} - Status: {order.get('status')}")
                
                return orders
            else:
                print(f"API Error {result.get('code')}: {result.get('message')}")
                return []
        else:
            print(f"HTTP Error {response.status_code}")
            try:
                error_data = response.json()
                print(f"   Error code: {error_data.get('code')}")
                print(f"   Error message: {error_data.get('message')}")
            except:
                print(f"   Raw response: {response.text}")
            return []
            
    except Exception as e:
        print(f"Exception in details: {e}")
        return []


In [58]:
# BƯỚC 8.3: Execute 2-Step Workflow
# ==============================================
# Mục đích: Orchestrate việc chạy Step 1 → Step 2 → Store results
# Input: ACCESS_TOKEN, SHOP_CIPHER từ các bước trước
# Output: REAL_ORDER_DETAILS (global variable chứa order data)
# ==============================================

print("BƯỚC 8.3: EXECUTING 2-STEP ORDER WORKFLOW...")
print("="*60)

# Execute 2-step workflow
if 'SHOP_CIPHER' in globals() and SHOP_CIPHER and 'ACCESS_TOKEN' in globals() and ACCESS_TOKEN:
    print("✅ Prerequisites OK: ACCESS_TOKEN and SHOP_CIPHER available")
    print("Starting 2-step order workflow...")
    
    # Step 1: Search for order IDs
    print("\n📋 STEP 1: SEARCHING FOR ORDER IDS...")
    order_ids = search_orders_for_ids()
    
    if order_ids:
        print(f"✅ Step 1 Complete! Found {len(order_ids)} order IDs")
        print(f"   Order IDs: {order_ids}")
        
        # Step 2: Get detailed order info
        print(f"\n📊 STEP 2: GETTING DETAILED ORDER INFO...")
        detailed_orders = get_order_details_with_ids(order_ids)
        
        if detailed_orders:
            print(f"✅ WORKFLOW SUCCESS! Retrieved {len(detailed_orders)} detailed orders")
            
            # Store results globally for other steps
            REAL_ORDER_DETAILS = detailed_orders
            print(f"\n💾 RESULTS STORED:")
            print(f"   Variable: REAL_ORDER_DETAILS")
            print(f"   Total orders: {len(detailed_orders)}")
            print(f"   Ready for analysis and export!")
            
            # Display sample order preview
            print(f"\n👀 SAMPLE ORDER PREVIEW:")
            sample = detailed_orders[0]
            key_fields = ['id', 'status', 'create_time', 'update_time']
            for field in key_fields:
                if field in sample:
                    value = sample[field]
                    if field in ['create_time', 'update_time'] and isinstance(value, int):
                        try:
                            readable_date = datetime.fromtimestamp(value).strftime('%Y-%m-%d %H:%M:%S')
                            print(f"   📅 {field}: {value} ({readable_date})")
                        except:
                            print(f"   📅 {field}: {value}")
                    else:
                        print(f"   📋 {field}: {value}")
            
        else:
            print(f"❌ Step 2 Failed: Could not get order details")
            print("   Check API permissions and Order IDs validity")
    else:
        print(f"❌ Step 1 Failed: Could not find order IDs")
        print("   Check filters, date range, or order status")
        
else:
    print("❌ PREREQUISITES MISSING!")
    if 'ACCESS_TOKEN' not in globals() or not ACCESS_TOKEN:
        print("   ⚠️  ACCESS_TOKEN not available - Run BƯỚC 6 first")
    if 'SHOP_CIPHER' not in globals() or not SHOP_CIPHER:
        print("   ⚠️  SHOP_CIPHER not available - Run BƯỚC 7 first")

print("="*60)

BƯỚC 8.3: EXECUTING 2-STEP ORDER WORKFLOW...
✅ Prerequisites OK: ACCESS_TOKEN and SHOP_CIPHER available
🚀 Starting 2-step order workflow...

📋 STEP 1: SEARCHING FOR ORDER IDS...
Step 1: Searching orders to get IDs...
Signature Debug:
   Path: /order/202309/orders/search
   Filtered params: {'app_key': '6h2cosrovhjab', 'timestamp': '1755348035', 'shop_cipher': 'ROW_lXdGlQAAAAAEW7t8sE8nkXPLwspQvX45', 'page_size': '20', 'sort_order': 'DESC', 'sort_field': 'create_time', 'sign_method': 'hmac_sha256'}
   Param string: app_key6h2cosrovhjabpage_size20shop_cipherROW_lXdGlQAAAAAEW7t8sE8nkXPLwspQvX45sign_methodhmac_sha256sort_fieldcreate_timesort_orderDESCtimestamp1755348035
   Body: {"create_time_ge":1747572035,"create_time_lt":1755348035}
   Sign string: /order/202309/orders/searchapp_key6h2cosrovhjabpage_size20shop_cipherROW_lXdGlQAAAAAEW7t8sE8nkXPLwspQvX45sign_methodhmac_sha256sort_fieldcreate_timesort_orderDESCtimestamp1755348035{"create_time_ge":1747572035,"create_time_lt":1755348035}
   W

In [59]:
# 📋 DISPLAY JSON DATA FROM STEP 8
# ================================
# Mục đích: Hiển thị dữ liệu JSON raw từ API để kiểm tra
# Input: REAL_ORDER_DETAILS từ Bước 8.3  
# Output: Formatted JSON display cho visual inspection
# ================================

print("📋 HIỂN THỊ DỮ LIỆU JSON TỪ BƯỚC 8...")
print("="*60)

if 'REAL_ORDER_DETAILS' in globals() and REAL_ORDER_DETAILS:
    print(f"✅ Có {len(REAL_ORDER_DETAILS)} đơn hàng để hiển thị")
    
    # Hiển thị JSON formatted cho tất cả đơn hàng
    print(f"\n📄 RAW JSON DATA:")
    print("-" * 50)
    
    import json
    
    # Pretty print JSON với indentation
    json_output = json.dumps(REAL_ORDER_DETAILS, indent=2, ensure_ascii=False)
    print(json_output)
    
    print("-" * 50)
    print(f"📊 SUMMARY:")
    print(f"   Total orders: {len(REAL_ORDER_DETAILS)}")
    print(f"   JSON size: {len(json_output)} characters")
    print(f"   Ready for BƯỚC 9 (Analysis) and BƯỚC 10 (Export)")
    
else:
    print("❌ KHÔNG CÓ DỮ LIỆU JSON")
    print("   Chạy BƯỚC 8.1-8.3 trước để lấy dữ liệu từ API")

print("="*60)

📋 HIỂN THỊ DỮ LIỆU JSON TỪ BƯỚC 8...
✅ Có 10 đơn hàng để hiển thị

📄 RAW JSON DATA:
--------------------------------------------------
[
  {
    "buyer_email": "v4bGVU2WK6N53Y5ENTPEBWLU3T52Y@scs2.tiktok.com",
    "buyer_message": "",
    "cancel_order_sla_time": 1755622799,
    "collection_due_time": 1755709199,
    "commerce_platform": "TIKTOK_SHOP",
    "create_time": 1755347595,
    "delivery_option_id": "7057025213938009858",
    "delivery_option_name": "Standard shipping",
    "delivery_type": "HOME_DELIVERY",
    "fulfillment_priority_level": 600,
    "fulfillment_type": "FULFILLMENT_BY_SELLER",
    "has_updated_recipient_address": false,
    "id": "580018118349457184",
    "is_cod": false,
    "is_on_hold_order": false,
    "is_replacement_order": false,
    "is_sample_order": false,
    "line_items": [
      {
        "currency": "VND",
        "display_status": "AWAITING_SHIPMENT",
        "id": "580018118349588256",
        "is_gift": false,
        "original_price": "1650000

In [ ]:
# BƯỚC 9: Analyze Order Data Structure
# ====================================
# Mục đích: Phân tích cấu trúc JSON data để hiểu các trường dữ liệu
# Input: REAL_ORDER_DETAILS từ Bước 8.3
# Output: Detailed analysis của data structure và content preview
# ====================================

print("BƯỚC 9: PHÂN TÍCH CẤU TRÚC DỮ LIỆU ĐƠN HÀNG...")
print("="*60)

if 'REAL_ORDER_DETAILS' in globals() and REAL_ORDER_DETAILS:
    print("Dataset Overview: {len(REAL_ORDER_DETAILS)} đơn hàng để phân tích")
    
    # Lấy sample để phân tích structure
    sample_order = REAL_ORDER_DETAILS[0]
    
    print(f"\nCẤU TRÚC DỮ LIỆU CHI TIẾT (Sample Order):")
    print("-" * 50)
    
    # Phân tích structure level 1
    total_fields = 0
    object_fields = 0
    array_fields = 0
    data_fields = 0
    
    for key, value in sample_order.items():
        total_fields += 1
        if isinstance(value, dict):
            object_fields += 1
            print(f"[Object] {key}: {len(value)} sub-fields")
        elif isinstance(value, list):
            array_fields += 1
            print(f"[Array] {key}: {len(value)} items")
        else:
            data_fields += 1
            value_preview = str(value)[:50] + "..." if len(str(value)) > 50 else str(value)
            print(f"[Data] {key}: {value_preview}")
    
    print(f"\nSTRUCTURE SUMMARY:")
    print(f"   Total fields: {total_fields}")
    print(f"   Object fields: {object_fields}")
    print(f"   Array fields: {array_fields}")
    print(f"   Data fields: {data_fields}")
    
    # Phân tích các section quan trọng
    print(f"\nPAYMENT INFORMATION:")
    if 'payment' in sample_order:
        payment = sample_order['payment']
        for key, value in payment.items():
            print(f"   {key}: {value}")
    else:
        print("   Payment info not available")
    
    print(f"\nPRODUCT INFORMATION:")
    if 'line_items' in sample_order:
        line_items = sample_order['line_items']
        print(f"   Total products: {len(line_items)}")
        for i, item in enumerate(line_items[:3]):  # Show first 3 products
            product_name = item.get('product_name', 'N/A')
            quantity = item.get('quantity', 'N/A')
            print(f"   Product {i+1}: {product_name} (Qty: {quantity})")
        if len(line_items) > 3:
            print(f"   ... and {len(line_items) - 3} more products")
    else:
        print("   Product info not available")
    
    print(f"\nSHIPPING INFORMATION:")
    if 'recipient_address' in sample_order:
        address = sample_order['recipient_address']
        print(f"   Recipient: {address.get('full_name', 'N/A')}")
        print(f"   Phone: {address.get('phone_number', 'N/A')}")
        print(f"   Address: {address.get('address_line1', 'N/A')}")
        print(f"   City: {address.get('city', 'N/A')}")
    else:
        print("   Shipping info not available")

else:
    print("NO DATA TO ANALYZE")
    print("   Run BƯỚC 8.1-8.3 first to get order data")

print("="*60)

BƯỚC 9: PHÂN TÍCH CẤU TRÚC DỮ LIỆU ĐƠN HÀNG...
📊 Dataset Overview: 10 đơn hàng để phân tích

🔍 CẤU TRÚC DỮ LIỆU CHI TIẾT (Sample Order):
--------------------------------------------------
📄 [Data] buyer_email: v4bGVU2WK6N53Y5ENTPEBWLU3T52Y@scs2.tiktok.com
📄 [Data] buyer_message: 
📄 [Data] cancel_order_sla_time: 1755622799
📄 [Data] collection_due_time: 1755709199
📄 [Data] commerce_platform: TIKTOK_SHOP
📄 [Data] create_time: 1755347595
📄 [Data] delivery_option_id: 7057025213938009858
📄 [Data] delivery_option_name: Standard shipping
📄 [Data] delivery_type: HOME_DELIVERY
📄 [Data] fulfillment_priority_level: 600
📄 [Data] fulfillment_type: FULFILLMENT_BY_SELLER
📄 [Data] has_updated_recipient_address: False
📄 [Data] id: 580018118349457184
📄 [Data] is_cod: False
📄 [Data] is_on_hold_order: False
📄 [Data] is_replacement_order: False
📄 [Data] is_sample_order: False
📋 [Array] line_items: 2 items
📄 [Data] order_type: NORMAL
📋 [Array] packages: 1 items
📄 [Data] paid_time: 1755347600
📂 [Object] payme

## BƯỚC 10: Export Order Data to Files

### 📁 Mục đích:
- **Transform JSON data** thành tabular format
- **Export ra multiple formats**: Excel (.xlsx) và CSV

In [ ]:
# BƯỚC 10: Export Order Data to Files
# ===================================
# Mục đích: Transform JSON data thành tabular format và export ra files
# Input: REAL_ORDER_DETAILS từ Bước 8.3
# Output: Excel file (.xlsx) và CSV file cho analysis và reporting
# ===================================

print("BƯỚC 10: XUẤT DỮ LIỆU RA FILE...")
print("="*60)

def export_orders_to_files():
    """
    Transform và export order data ra Excel và CSV files với MULTIPLE-ROW STRATEGY
    
    NEW Transformation logic:
    - 1 Order có N line_items → N rows trong output
    - Mỗi row = Order info + 1 Product info + Package info (duplicate)
    - DATA FIELDS: preserved as-is
    - OBJECT FIELDS: flattened với [sub-field] naming (payment[total_amount])
    - ARRAY FIELDS: multiple rows (1 product = 1 row)
    """
    
    if 'REAL_ORDER_DETAILS' not in globals() or not REAL_ORDER_DETAILS:
        print("❌ NO DATA TO EXPORT")
        print("   Run BƯỚC 8.1-8.3 first to get order data")
        return None, None
    
    try:
        print(f"🔄 TRANSFORMING {len(REAL_ORDER_DETAILS)} orders to MULTIPLE-ROW format...")
        
        # Transform JSON to multiple-row table structure
        export_data = []
        total_products = 0
        
        for order in REAL_ORDER_DETAILS:
            # 📋 Extract basic order information (DATA FIELDS) - COMPLETE VERSION
            order_base = {
                'id': order.get('id'),
                'status': order.get('status'),
                'create_time': datetime.fromtimestamp(order.get('create_time', 0)).strftime('%Y-%m-%d %H:%M:%S'),
                'update_time': datetime.fromtimestamp(order.get('update_time', 0)).strftime('%Y-%m-%d %H:%M:%S'),
                'commerce_platform': order.get('commerce_platform'),
                'delivery_type': order.get('delivery_type'),
                'fulfillment_type': order.get('fulfillment_type'),
                'warehouse_id': order.get('warehouse_id'),
                'is_cod': order.get('is_cod'),
                'is_buyer_request_cancel': order.get('is_buyer_request_cancel'),
                'cancel_reason': order.get('cancel_reason'),
                'cancel_user': order.get('cancel_user'),
                'is_replacement_order': order.get('is_replacement_order'),
                'buyer_email': order.get('buyer_email'),
                'buyer_message': order.get('buyer_message'),
                'buyer_uid': order.get('buyer_uid'),
                'split_or_combine_tag': order.get('split_or_combine_tag'),
                'seller_note': order.get('seller_note'),
                'prescription_images': order.get('prescription_images'),
                'prescription_videos': order.get('prescription_videos'),
                'delivery_option': order.get('delivery_option'),
                'delivery_due_time': order.get('delivery_due_time'),
                'is_on_hold_order': order.get('is_on_hold_order'),
                'collection_due_time': order.get('collection_due_time'),
                'tracking_number': order.get('tracking_number'),
                'shipping_provider': order.get('shipping_provider'),
                'shipping_provider_id': order.get('shipping_provider_id'),
                'user_id': order.get('user_id'),
                'paid_time': order.get('paid_time'),
                'rts_time': order.get('rts_time'),
                'rts_sla_time': order.get('rts_sla_time'),
                'delivery_sla_time': order.get('delivery_sla_time'),
                'collection_sla_time': order.get('collection_sla_time'),
                'order_line_id': order.get('order_line_id'),
                'cpf': order.get('cpf'),
                # ADD MORE FIELDS từ JSON
                'cancel_order_sla_time': order.get('cancel_order_sla_time'),
                'delivery_option_id': order.get('delivery_option_id'),
                'delivery_option_name': order.get('delivery_option_name'),
                'fulfillment_priority_level': order.get('fulfillment_priority_level'),
                'has_updated_recipient_address': order.get('has_updated_recipient_address'),
                'is_sample_order': order.get('is_sample_order'),
                'order_type': order.get('order_type'),
                'payment_method_name': order.get('payment_method_name'),
                'recommended_shipping_time': order.get('recommended_shipping_time'),
                'shipping_due_time': order.get('shipping_due_time'),
                'shipping_type': order.get('shipping_type'),
                'tts_sla_time': order.get('tts_sla_time'),
            }
            
            # 💰 Payment information (OBJECT FIELD → flattened với [sub-field] naming)
            payment = order.get('payment', {})
            payment_flat = {f'payment[{k}]': v for k, v in payment.items()}
            order_base.update(payment_flat)
            
            # 🏠 Address information (OBJECT FIELD → flattened với [sub-field] naming)
            address = order.get('recipient_address', {})
            address_flat = {f'recipient_address[{k}]': v for k, v in address.items()}
            order_base.update(address_flat)
            
            # 📦 Package information (ARRAY FIELD → sẽ duplicate cho mỗi product)
            packages = order.get('packages', [])
            package_info = {}
            if packages:
                # Lấy package đầu tiên (thường chỉ có 1 package per order)
                package = packages[0]
                # CHỈ LẤY CÁC FIELDS THỰC SỰ CÓ TRONG JSON
                package_info = {
                    'package_id': package.get('id'),
                    'package_shipping_provider': package.get('shipping_provider'),
                    'package_shipping_provider_id': package.get('shipping_provider_id'),
                    'package_tracking_number': package.get('tracking_number'),
                }
            
            # 🛍️ Product information (ARRAY FIELD → MULTIPLE ROWS)
            line_items = order.get('line_items', [])
            
            if line_items:
                # Tạo 1 row cho mỗi product
                for item in line_items:
                    row = order_base.copy()  # Copy all order info
                    row.update(package_info)  # Add package info (duplicate for all products)
                    
                    # Add product-specific info - CONSISTENT [sub-field] naming
                    row.update({
                        # Fields từ line_items JSON với [sub-field] format
                        'lineitem[currency]': item.get('currency'),
                        'lineitem[display_status]': item.get('display_status'),
                        'lineitem[id]': item.get('id'),
                        'lineitem[is_gift]': item.get('is_gift'),
                        'lineitem[original_price]': item.get('original_price'),
                        'lineitem[package_id]': item.get('package_id'),
                        'lineitem[package_status]': item.get('package_status'),
                        'lineitem[platform_discount]': item.get('platform_discount'),
                        'lineitem[product_id]': item.get('product_id'),
                        'lineitem[product_name]': item.get('product_name'),
                        'lineitem[sale_price]': item.get('sale_price'),
                        'lineitem[seller_discount]': item.get('seller_discount'),
                        'lineitem[seller_sku]': item.get('seller_sku'),
                        'lineitem[shipping_provider_id]': item.get('shipping_provider_id'),
                        'lineitem[shipping_provider_name]': item.get('shipping_provider_name'),
                        'lineitem[sku_id]': item.get('sku_id'),
                        'lineitem[sku_image]': item.get('sku_image'),
                        'lineitem[sku_name]': item.get('sku_name'),
                        'lineitem[sku_type]': item.get('sku_type'),
                        'lineitem[tracking_number]': item.get('tracking_number'),
                        # Legacy fields để compatibility (có thể remove sau)
                        'product_id': item.get('product_id'),
                        'product_name': item.get('product_name'),
                        'sku_id': item.get('sku_id'),
                        'sku_name': item.get('sku_name'),
                        'sku_image': item.get('sku_image'),
                        'quantity': item.get('quantity', 1),  # Default 1 nếu không có
                        'seller_sku': item.get('seller_sku'),
                        'sku_type': item.get('sku_type'),
                        'is_gift': item.get('is_gift'),
                        'display_price': item.get('sale_price'),  # Use sale_price as display_price
                        'platform_discount': item.get('platform_discount'),
                        'seller_discount': item.get('seller_discount'),
                        # CALCULATED FIELDS
                        'total_product_price': (item.get('quantity', 1) * float(item.get('sale_price', 0))) if item.get('sale_price') else 0,
                    })
                    
                    export_data.append(row)
                    total_products += 1
            else:
                # Order không có products → vẫn tạo 1 row với order info
                row = order_base.copy()
                row.update(package_info)
                export_data.append(row)
        
        # 📊 Create DataFrame for export
        df = pd.DataFrame(export_data)
        
        # 📁 Generate timestamped filenames
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        excel_filename = f"TikTok_Orders_MultiRow_{timestamp}.xlsx"
        csv_filename = f"TikTok_Orders_MultiRow_{timestamp}.csv"
        
        # 📈 Export to Excel
        print(f"📈 Exporting to Excel...")
        df.to_excel(excel_filename, index=False, engine='openpyxl')
        print(f"   Excel exported: {excel_filename}")
        
        # Export to CSV
        print(f"Exporting to CSV...")
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"   CSV exported: {csv_filename}")
        
        # 📊 Export summary cho MULTIPLE-ROW format
        unique_orders = df['id'].nunique() if 'id' in df.columns else 0
        print(f"\n📊 MULTIPLE-ROW EXPORT SUMMARY:")
        print(f"   📦 Unique orders processed: {unique_orders}")
        print(f"   🛍️  Total products (rows): {len(export_data)}")
        print(f"   📋 Total columns: {len(df.columns)}")
        print(f"   📈 Avg products per order: {total_products/unique_orders:.1f}")
        print(f"   📁 Files saved to: Current directory")
        print(f"   📈 Excel file: {excel_filename}")
        print(f"   📋 CSV file: {csv_filename}")
        
        # 👀 Data preview với meaningful columns cho multiple-row
        print(f"\n👀 DATA PREVIEW (Multiple-Row Format):")
        if len(df) > 0:
            preview_cols = []
            # Chọn columns có ý nghĩa để preview
            possible_cols = ['id', 'status', 'lineitem[product_name]', 'quantity', 'payment[total_amount]', 'recipient_address[full_name]']
            for col in possible_cols:
                if col in df.columns:
                    preview_cols.append(col)
            
            if preview_cols:
                print(df[preview_cols].head(8).to_string())  # Show 8 rows để thấy multiple products
            else:
                print(df.head(3).to_string())
        
        return excel_filename, csv_filename
        
    except Exception as e:
        print(f"❌ EXPORT ERROR: {e}")
        print("   Check pandas and openpyxl installation")
        return None, None

# 🚀 Execute export process
print("🚀 STARTING MULTIPLE-ROW EXPORT PROCESS...")
if 'REAL_ORDER_DETAILS' in globals() and REAL_ORDER_DETAILS:
    excel_file, csv_file = export_orders_to_files()
    
    if excel_file and csv_file:
        print(f"\n🎉 MULTIPLE-ROW EXPORT COMPLETED SUCCESSFULLY!")
        print(f"   📈 Excel: {excel_file}")
        print(f"   📋 CSV: {csv_file}")
        print(f"\n💡 FORMAT EXPLANATION:")
        print(f"   • 1 Order với N products → N rows trong file")

else:
    print("❌ NO DATA AVAILABLE FOR EXPORT")
    print("   Please run BƯỚC 8.1-8.3 first to retrieve order data")

print("="*60)

BƯỚC 10: XUẤT DỮ LIỆU RA FILE...
🚀 STARTING MULTIPLE-ROW EXPORT PROCESS...
🔄 TRANSFORMING 10 orders to MULTIPLE-ROW format...
📈 Exporting to Excel...
   ✅ Excel exported: TikTok_Orders_MultiRow_20250816_194037.xlsx
📋 Exporting to CSV...
   ✅ CSV exported: TikTok_Orders_MultiRow_20250816_194037.csv

📊 MULTIPLE-ROW EXPORT SUMMARY:
   📦 Unique orders processed: 10
   🛍️  Total products (rows): 15
   📋 Total columns: 111
   📈 Avg products per order: 1.5
   📁 Files saved to: Current directory
   📈 Excel file: TikTok_Orders_MultiRow_20250816_194037.xlsx
   📋 CSV file: TikTok_Orders_MultiRow_20250816_194037.csv

👀 DATA PREVIEW (Multiple-Row Format):
                   id             status                                                                                               lineitem[product_name]  quantity payment[total_amount]
0  580018118349457184  AWAITING_SHIPMENT                                                       Vợt Pickleball Facolos Sport Series (Colorful Collection 16MM)  